In [32]:
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [33]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
def cleanTweet(tweet: str) -> str:
    tweet = re.sub(r'http\S+', '', str(tweet))
    tweet = re.sub(r'bit.ly/\S+', '', str(tweet))
    tweet = tweet.strip('[link]')

    # remove users
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', str(tweet))
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', str(tweet))

    # remove puntuation
    my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@â'
    tweet = re.sub('[' + my_punctuation + ']+', ' ', str(tweet))

    # remove number
    tweet = re.sub('([0-9]+)', '', str(tweet))

    # remove hashtag
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', str(tweet))

    return tweet


In [45]:
 spark = SparkSession\
        .builder\
        .appName("TwitterSentimentAnalysis")\
        .master("local[*]")\
        .getOrCreate()

In [46]:
spark.sparkContext.setLogLevel('ERROR')

In [47]:
df = spark.readStream\
        .format("socket")\
        .option("host", "127.0.0.1")\
        .option("port", 3333)\
        .load()

In [48]:
tweet_schema = StructType().add("ID", "string").add("text", "string").add("created_at", "string")

In [49]:
values = df.select(from_json(df.value.cast("string"), tweet_schema).alias("tweet"))

In [51]:
df1 = values.select("tweet.*")
clean_tweets = F.udf(cleanTweet, StringType())
raw_tweets = df1.withColumn('processed_text', clean_tweets(col("text")))

In [52]:
writeTweet = raw_tweets.writeStream. \
    outputMode("append"). \
    format("console"). \
    queryName("tweetquery"). \
    trigger(processingTime='2 seconds'). \
    start()


[Stage 0:>                                                          (0 + 8) / 8]



-------------------------------------------
Batch: 0
-------------------------------------------
+---+----+----------+--------------+
| ID|text|created_at|processed_text|
+---+----+----------+--------------+
+---+----+----------+--------------+

-------------------------------------------
Batch: 1
-------------------------------------------
+-------------------+--------------------+--------------------+--------------------+
|                 ID|                text|          created_at|      processed_text|
+-------------------+--------------------+--------------------+--------------------+
|1593497859730690048|RT @chartdata_twt...|2022-11-18 06:54:...|rt     was named ...|
+-------------------+--------------------+--------------------+--------------------+



In [44]:
spark.stop()
